In [6]:
import pymysql
import os
import zipfile  
import csv
import sys
import logging
import time
import datetime
import shutil
import numpy as np
from openpyxl.reader.excel import load_workbook
num = 0
mfm_count = []
vdr_count = []
kyma_count = []
mfm_date = []
vdr_date = []
kyma_date = []
start_time = time.time()
db = pymysql.connect(host='localhost', user='root', passwd='', db='ymupsurgence', charset='utf8')  #連接資料庫    
cursor = db.cursor()
path = "D:\\桌面\\ship\\ship"     #raw data 存放的資料夾
for root,dirs,files in os.walk(path): 
    for name in files:
        if name.endswith('.txt'):
            filename = open(os.path.join(root, name),'r') 
            lines = filename.readlines() 
            num += 1
            if(num % 50 == 0):
                print(num)
                print(name)
           
            #mfm 檔案處理
            if name.startswith('MFM'):
                for line in lines:
                    line = line.strip('\n')
                    line = line.split(',')
                    if(len(line[0]) == 19):
                        if(int(line[0][15])%5 == 0 and line[0][17] == '0'):
                            current_date = line[0][0:10]
                            if(current_date not in mfm_date):
                                mfm_date.append(current_date)
                                mfm_count.append([])
                                for i in range(0,288):
                                    mfm_count[-1].append(0)
                                date_index = len(mfm_count)-1
                            else:
                                date_index = mfm_date.index(current_date)
                            time_index = int(int(line[0][15])/5 + int(line[0][14])*2 + int(line[0][12])*12 + int(line[0][11])*120)
                            mfm_count[date_index][time_index] += 1
                            results = mfm_count[date_index][time_index]-1
                            if(results >= 50):
                                print("mfm",mfm_date[date_index],time_index)
                        else:
                            continue
                    else:
                        continue
                    #若無相同時間，則用insert
                    if results == 0:
                        insert = "INSERT INTO mfm(DateTime,%s) \
                                VALUES ('%s','%s')" % \
                                (line[1],line[0],line[3])
                        try:    
                            cursor.execute(insert)
                            
                        except:                            
                            db.rollback()
                            print("insert mfm error")                            
                    #若已存在相同時間，改用update
                    else:
                        update = "UPDATE mfm SET %s = '%s'WHERE DateTime = '%s' " %(line[1],line[3],line[0])
                        try:
                            cursor.execute(update)
                        except:
                            db.rollback()                                      
                            print("update mfm error")
                
                db.commit()
                filename.close()
            #vdr 檔案處理
            elif name.startswith('SI'):
                for line in lines:
                    line = line.strip('\n')
                    line = line.split(',')
                    '''
                    if(len(line[0]) == 19):
                        if(int(line[0][15])%5 == 0 and line[0][17] == '0'):
                            current_date = line[0][0:10]
                            if(current_date not in vdr_date):
                                vdr_date.append(current_date)
                                vdr_count.append([])
                                for i in range(0,288):
                                    vdr_count[-1].append(0)
                                date_index = len(vdr_count)-1
                            else:
                                date_index = vdr_date.index(current_date)
                            time_index = int(int(line[0][15])/5 + int(line[0][14])*2 + int(line[0][12])*12 + int(line[0][11])*120)
                            vdr_count[date_index][time_index] += 1
                            results = vdr_count[date_index][time_index]-1
                            if(results >= 32):
                                print("vdr",vdr_date[date_index],time_index)
                        else:
                            continue
                    else:
                        continue
                    '''
                    if(len(line[0]) == 19):
                        if(int(line[0][15])%5 == 0 and line[0][17] == '0'):
                            pass
                        else:
                            continue
                    else:
                        continue      
                    select = "SELECT * FROM vdr WHERE DateTime = '%s' " %(line[0])
                    try:
                        cursor.execute(select)
                        results = cursor.rowcount
                    except:
                        db.rollback()
                    #SI00 檔名處理
                    if name.startswith('SI00'):
                        # latitude, longitude 欄位處理
                        if line[1] == 'GPGGA':                            
                            if results == 0:                                
                                insert = "INSERT INTO vdr (DateTime, `latitude`, `longitude`) \
                                        VALUES ('%s','%s','%s')" % \
                                        (line[0],line[3]+line[4],line[5]+line[6])
                                try:                             
                                    cursor.execute(insert)
                                except:                                    
                                    db.rollback()                             
                            else:
                                update = "UPDATE vdr SET `latitude` = '%s',`longitude` = '%s'WHERE DateTime = '%s' " %(line[3]+line[4],line[5]+line[6],line[0])
                                try:
                                    cursor.execute(update)
                                except:
                                    db.rollback()
                        # course, ground speed 欄位處理
                        elif line[1] == 'GPRMC':
                            if results == 0:
                                insert = "INSERT INTO vdr(DateTime,`course`,`ground speed`) \
                                        VALUES ('%s','%s','%s')" % \
                                        (line[0],line[9],line[8])
                                try:
                                    cursor.execute(insert)                               
                                except:                                   
                                    db.rollback()                          
                            else:
                                update = "UPDATE vdr SET `course` = '%s', `ground speed` = '%s' WHERE DateTime = '%s' " %(line[9],line[8],line[0])
                                try:
                                    cursor.execute(update)
                                except:
                                    db.rollback()
                    #SI01 檔名處理
                    elif name.startswith('SI01'):
                        # rudder angle 欄位處理
                        if line[1] == 'IIRSA':
                            if results == 0:
                                insert = "INSERT INTO vdr(DateTime,`rudder angle`) \
                                        VALUES ('%s','%s')" % \
                                        (line[0],line[2])
                                try:           
                                    cursor.execute(insert)             
                                except:                 
                                    db.rollback()        
                            else:
                                update = "UPDATE vdr SET `rudder angle` = '%s'WHERE DateTime = '%s' " %(line[2],line[0])
                                try:
                                    cursor.execute(update)
                                except:
                                    db.rollback()                                    
                    #SI02 檔名處理                    
                    elif name.startswith('SI02'):
                        # wind speed, wind angle 欄位處理
                        if line[1] == 'WIMWV':
                            if results == 0:
                                insert = "INSERT INTO vdr(DateTime,`wind speed`,`wind angle`) \
                                        VALUES ('%s','%s','%s')" % \
                                        (line[0],line[4],line[2])
                                try:                                
                                    cursor.execute(insert)                               
                                except:                                   
                                    db.rollback()                            
                            else:
                                update = "UPDATE vdr SET `wind speed` = '%s', `wind angle` = '%s'WHERE DateTime = '%s' " %(line[4],line[2],line[0])
                                try:
                                    cursor.execute(update)
                                except:
                                    db.rollback()
                    #SI03 檔名處理              
                    elif name.startswith('SI03'):
                        # water speed 欄位處理                        
                        if line[1] == 'VDVBW':
                            if results == 0:
                                insert = "INSERT INTO vdr(DateTime,`water speed`) \
                                        VALUES ('%s','%s')" % \
                                        (line[0],line[2])
                                try:            
                                    cursor.execute(insert)            
                                except:                
                                    db.rollback()        
                            else:
                                update = "UPDATE vdr SET `water speed` = '%s' WHERE DateTime = '%s' " %(line[2],line[0])
                                try:
                                    cursor.execute(update)
                                except:
                                    db.rollback()                    
                    #SI04 檔名處理
                    elif name.startswith('SI04'):
                        # heading 欄位處理                        
                        if line[1] == 'HEHDT': 
                            if results == 0:
                                insert = "INSERT INTO vdr(DateTime,`heading`) \
                                        VALUES ('%s','%s')" % \
                                        (line[0],line[2])
                                try:                                
                                    cursor.execute(insert)                                
                                except:                                    
                                    db.rollback()            
                            else:
                                update = "UPDATE vdr SET `heading` = '%s' WHERE DateTime = '%s' " %(line[2],line[0])
                                try:
                                    cursor.execute(update)
                                except:
                                    db.rollback()               
                    #SI05 檔名處理
                    elif name.startswith('SI05'):
                        # depth 欄位處理
                        if line[1] == 'SDDPT':
                            if results == 0:
                                insert = "INSERT INTO vdr(DateTime,`depth`) \
                                        VALUES ('%s','%s')" % \
                                        (line[0],line[2])
                                try:          
                                    cursor.execute(insert)               
                                except:                    
                                    db.rollback()          
                            else:
                                update = "UPDATE vdr SET `depth` = '%s'WHERE DateTime = '%s' " %(line[2],line[0])
                                try:
                                    cursor.execute(update)
                                except:
                                    db.rollback()                   
                    #SI07 檔名處理                   
                    elif name.startswith('SI07'):
                        # revolutions 欄位處理
                        if line[1] == 'RCRPM':
                            if results == 0:
                                insert = "INSERT INTO vdr(DateTime,revolutions) \
                                        VALUES ('%s','%s')" % \
                                        (line[0],line[4])
                                try:                               
                                    cursor.execute(insert)                               
                                except:                                   
                                    db.rollback()                           
                            else:
                                update = "UPDATE vdr SET revolutions = '%s'WHERE DateTime = '%s' " %(line[4],line[0])
                                try:
                                    cursor.execute(update)
                                except:
                                    db.rollback()
                db.commit()
                filename.close()
                                
                            
                            
            #KYMA 檔案處理
            elif name.startswith('SA'):
                for line in lines:
                    line = line.strip('\n')
                    line = line.split(',')
                    if line[1] == 'SA8I':
                        if(len(line[0]) == 19):
                            if(int(line[0][15])%5 == 0 and line[0][17] == '0'):
                                current_date = line[0][0:10]
                                if(current_date not in kyma_date):
                                    kyma_date.append(current_date)
                                    kyma_count.append([])
                                    for i in range(0,288):
                                        kyma_count[-1].append(0)
                                    date_index = len(kyma_count)-1
                                else:
                                    date_index = kyma_date.index(current_date)
                                time_index = int(int(line[0][15])/5 + int(line[0][14])*2 + int(line[0][12])*12 + int(line[0][11])*120)
                                kyma_count[date_index][time_index] += 1
                                results = kyma_count[date_index][time_index]-1
                                if(results > 1):
                                    print("kyma",kyma_date[date_index],time_index)
                            else:
                                continue
                        else:
                            continue

                        if results == 0:
                            
                            insert = "INSERT INTO kyma (DateTime, `torque`, `thrust`, `engine speed`, `power`, `energy`, `total revolutions`) \
                                    VALUES ('%s','%s','%s','%s','%s','%s','%s')" % \
                                    (line[0],line[2],line[3],line[4],line[5],line[6],line[7])
                            try:
                            
                                cursor.execute(insert)
                            except:                                
                                db.rollback()
                                print("insert kyma error")                                
                        else:

                            update = "UPDATE kyma SET `torque` = '%s',`thrust` = '%s', `engine speed` = '%s', `power` = '%s', `energy` = '%s', `total revolutions` = '%s' \
                                      WHERE DateTime = '%s' " %(line[2],line[3],line[4],line[5],line[6],line[7], line[0])
                            try:
                                cursor.execute(update)
                            except:
                                db.rollback()
                                print("update kyma error")
                                
                db.commit()
                filename.close() 
db.close ()
end_time = time.time()-start_time
print("All Done In = " + str(end_time) + "s") 


50
MFM-20171101-AI1060.txt
100
MFM-20171102-AI1060.txt
All Done In = 8.8905770778656s
